In [1]:
import numpy as np
import constants

In [86]:
T = constants.T  # number of minutes in a shift
N = constants.N  # number of resource categories
M = constants.M  # number of shifts
p = constants.p  # patients matrix per resource per shift
x = constants.x  # staff availability matrix
t = constants.t  # time for resource matrix

ARRIVAL_TIMES = constants.ARRIVAL_TIMES


In [99]:
def simulation_by_shift(i, j, x, t, o):
    """
    :param i: index of resource [0, 8]
    :param j: index of shift [0, 2]
    :param x: number of staff assigned to resource i in shift j
    :param t: number of minutes it takes for resource i in shift j to serve 1 patient
    :param o: number of patients overflow from previous shifts for resource i in shift j

    :return: number of patients still waiting 
    """
    cur_waiting = o
    staff_availability = dict([x, 0] for x in range(x)) # dict to hold minutes until free for each staff member

    for minute in range(j*T, (j+1)*T):

        if minute in ARRIVAL_TIMES[i]:
            cur_waiting += 1

        x_available = [k for k, v in staff_availability.items() if v == 0] # keys of staff that are free
        if len(x_available) == 0 or minute + t > (j+1)*T:  # if no available staff or not enough time remaining in shift
            pass 
        elif cur_waiting <= len(x_available):          # if enough staff to attend to number of patients waiting
            for k in range(cur_waiting):
                staff_availability[x_available[k]] = t
                served_times[i].append(minute)
            cur_waiting = 0
        else:                                          # if less staff available than waiting patients
            for k in x_available:
                staff_availability[k] = t       
                served_times[i].append(minute)
            cur_waiting -= len(x_available)
            
        # decrement minutes until free at the end of each loop
        for k in staff_availability.keys():
            staff_availability[k] = max(0, staff_availability[k]-1)

    return cur_waiting


def simulation():
    """Runs simulation per shift for each resource"""
    
    served_times = [[] for _ in range(N)]
    
    for i in range(N):
        o_1 = simulation_by_shift(i, 0, x[i][0], t[i][0], 0)
        o_2 = simulation_by_shift(i, 1, x[i][1], t[i][1], o_1)
        o_3 = simulation_by_shift(i, 2, x[i][2], t[i][2], o_2)

        if o_3 != 0:
            return np.inf

        wait_times = [np.array(served_times[i]) - np.array(ARRIVAL_TIMES[i]) for i in range(N)]
        
        return np.sum([np.sum(wait_times[i]) for i in range(N)])


def calculate_objective(x, c, d):
    """
    :param x: matrix of number of staff working for each resource in each shift
    :param c: matrix of costs for each resource in each shift
    :param d: cost for a unit of wait time

    """
    resource_cost = sum([c[i][j]*x[i][j] for j in range(M) for i in range(N)])
    wait_cost = simulation()

    return resource_cost + wait_cost
            
# TODO: make sure constraints are satisfied
# TODO: add c and d to constants file     

In [100]:
simulation()

In [98]:
served_times = [[] for _ in range(N)]

In [ ]:
[len(served_times[i]) for i in range(N)]

In [109]:
test = [np.array(served_times[i]) - np.array(ARRIVAL_TIMES[i]) for i in range(N)]
np.sum([np.sum(test[i]) for i in range(N)])